In [ ]:
import numpy as np
import scipy.stats as st

def calculate_p_value_from_z_score(score):
    p_value = st.t.sf((abs(score)), 100000) * 2
    return min(float(p_value), 0.99999)

#Cohens Kappa fpr two raters
contingency_table = np.array([[106,10,4],[22,28,10],[2,12,6]])
contingency_table = np.array([[4,4,2],[2,6,0],[0,2,0]])
#contingency_table = np.array([[10,4,1],[6,16,2],[0,3,8]])
#contingency_table2 = np.array([[20,5],[10,15]])

confidence_level = 0.95

def Cohens_Kappa_2_raters(contingency_table, confidence_level):
    Sample_Size = np.sum(contingency_table)
    Diagonal_of_Agreement = np.diag(contingency_table)
    Sum_of_coloumns = np.sum(contingency_table, axis=0) / Sample_Size
    Sum_of_rows = np.sum(contingency_table, axis=1) / Sample_Size
    Disagreement_Table = np.outer(Sum_of_coloumns, Sum_of_rows)
    Percentages_of_DissAgreement = np.trace(Disagreement_Table)
    Percentages_of_Agreement = np.sum((Diagonal_of_Agreement)/ Sample_Size)
    Cohens_Kappa = (Percentages_of_Agreement - Percentages_of_DissAgreement) / (1 - Percentages_of_DissAgreement)

    # Calculate the Variance (Fleiss, Cohen & Everrit, 1969)
    number_of_levels = contingency_table.shape[1]
    Agreement_Matrix = np.eye(contingency_table.shape[1])
    Probabilty_Matrix = (contingency_table/Sample_Size)
    Probability_Coloumns = np.repeat(np.sum(Probabilty_Matrix, axis=1) * (1-Cohens_Kappa), repeats=number_of_levels)
    Probability_Rows = np.repeat((np.sum(Probabilty_Matrix, axis=0) * (1 - Cohens_Kappa))[np.newaxis, :], repeats=number_of_levels, axis=0)
    Final_Probabilities = np.transpose(Probabilty_Matrix).flatten() * (Agreement_Matrix.flatten() -  Probability_Coloumns - Probability_Rows.flatten())**2
    Standart_Error_Kappa_Fleisse = np.sqrt((np.sum(Final_Probabilities) - (Cohens_Kappa - Percentages_of_DissAgreement * (1 - Cohens_Kappa))**2) / (1-Percentages_of_DissAgreement)**2 / Sample_Size)

    # Calculate the Variance for H0 (Fleiss, Cohen & Everrit, 1969)
    outer_mulitiplication_matrix = np.transpose(np.outer(Sum_of_rows, Sum_of_coloumns))
    outer_addition_matrix = np.add.outer(Sum_of_rows, Sum_of_coloumns)
    Variance_matrix = outer_mulitiplication_matrix * (Agreement_Matrix - outer_addition_matrix)**2
    Standard_Error_H0 = np.sqrt((np.sum(Variance_matrix) - Percentages_of_DissAgreement**2) / (Sample_Size * (1-Percentages_of_DissAgreement)**2))

    # Signficance
    StatisticH0 = Cohens_Kappa / Standard_Error_H0
    p_valueH0 = calculate_p_value_from_z_score(StatisticH0)
    Statistic_Fleiss = Cohens_Kappa / Standart_Error_Kappa_Fleisse
    p_value_Fleiss = calculate_p_value_from_z_score(Statistic_Fleiss)

    # Confidence Interval
    zcrit = st.t.ppf(1 - (1 - confidence_level) / 2, 100000)
    Lower_Confidence_Interval_Kappa = Cohens_Kappa - Standart_Error_Kappa_Fleisse*zcrit
    Upper_Confidence_Interval_Kappa = Cohens_Kappa + Standart_Error_Kappa_Fleisse*zcrit
    Lower_Confidence_Interval_Kappa_H0 = Cohens_Kappa - Standard_Error_H0*zcrit
    Upper_Confidence_Interval_Kappa_H0 = Cohens_Kappa + Standard_Error_H0*zcrit


    results = {}

    results["Cohens Kappa"]= Cohens_Kappa
    results["p_value"] = p_valueH0
    results["Statistic"]= StatisticH0
    results["Standard_Error_H0"]= Standard_Error_H0
    results["Standart_Error_Fleisse, Cohen & Everett"] = Standart_Error_Kappa_Fleisse
    results["p_value (Fleisse, Cohen & Everett)"] = p_value_Fleiss
    results["Statistic (Fleisse, Cohen & Everett)"]= Statistic_Fleiss
    results["Confidence Intervals Kappa"] = f"({round(Lower_Confidence_Interval_Kappa, 4)}, {round(Upper_Confidence_Interval_Kappa, 4)})"
    results["Confidence Intervals Kappa_H0"] = f"({round(Lower_Confidence_Interval_Kappa_H0, 4)}, {round(Upper_Confidence_Interval_Kappa_H0, 4)})"

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return result_str

a = Cohens_Kappa_2_raters(contingency_table, confidence_level)
#b = Cohens_Kappa_2_raters(contingency_table2, confidence_level)

print(a)
